In [1]:
import pandas as pd
import numpy as np

df_John = pd.read_excel('/Users/guapofish/Desktop/Deer Isle Group/John Niederer Part/DeerIsle ABWebsites062320.xlsx')
data = pd.read_excel("/Users/guapofish/Desktop/Deer Isle Group/Python/data.xlsx")
key = pd.read_csv('/Users/guapofish/Desktop/Deer Isle Group/Python/matching.csv')
info = data[key.columns]

In [2]:
#This get_mid function was written by Lawrance, which chose only one company with 
#the same CRD number within United States left for further data cleaning 
def get_mid(info):
    code = info['FirmCrdNb'].unique()
    #len(code) #18084

    d = []
    for i in range(len(code)):
        d.append(info[info.FirmCrdNb == code[i]].iloc[len(info[info.FirmCrdNb == code[i]])-1])
    
    d = pd.DataFrame(d)
    d = d[d.Cntry == 'United States']
    return d.sort_values(by='FirmCrdNb', ascending=True).reset_index(drop=True)

In [107]:
def get_concatCRD_John(f,name):
    
    def get_John_CRD(x):
        crd = np.array(x['CRDNumber']).astype(int)  
        crd = np.array(crd).astype(str)
        x['CRDNumber'].iloc[0] = ''.join(crd)
        return x

    f = f.groupby(name).apply(lambda x:get_John_CRD(x))
    f = f.groupby(name).first()
    return f.reset_index()


def get_concatCRD_mid(f,name):
    
    def get_mid_CRD(x):
        crd = np.array(x['FirmCrdNb']).astype(int)  
        crd = np.array(crd).astype(str)
        x['FirmCrdNb'].iloc[0] = ''.join(crd)
        return x
    
    f = f.groupby(name).apply(lambda x:get_mid_CRD(x))
    f = f.groupby(name).first()
    return f.reset_index()


def get_additional_col(mid,name_list):
    
    dic = {'Q5DA1': 'Individuals' , 'Q5DA3': 'Individuals_AUM', 
     'Q5DB1':'High net worth individuals', 'Q5DB3':'High net worth individual_AUM',
     'Q5DC1':'Banking or thrift institutions', 'Q5DC3':'Banking or thrift institution_AUM',
     'Q5DD1':'Investment companies', 'Q5DD3':'Investment companies_AUM',
     'Q5DE1':'Business development companies', 'Q5DE3':'Business development companies_AUM',
     'Q5DF1':'Pooled investment vehicles', 'Q5DF3':'Pooled investment vehicles_AUM',
     'Q5DG1':'Pension and profit sharing plans', 'Q5DG3':'Pension and profit sharing plans_AUM',
     'Q5DH1':'Charitable organizations', 'Q5DH3':'Charitable organizations_AUM',
     'Q5DI1':'State or municipal government entities', 'Q5DI3':'State or municipal government entities_AUM',
     'Q5DJ1':'Other investment advisers', 'Q5DJ3':'Other investment advisers_AUM',
     'Q5DK1':'Insurance companies', 'Q5DK3':'Insurance companies_AUM',
     'Q5DL1':'Sovereign wealth funds and foreign official institutions', 'Q5DL3':'Sovereign wealth funds and foreign official institutions_AUM',
     'Q5DM1':'Corporations or other businesses not listed above', 'Q5DM3':'Corporations or other businesses not listed above_AUM',
     'Q5DN1':'Other', 'Q5DN3':'Other_AUM'}

    m = mid[mid.columns[28:56]]
    m = m.fillna(0)
    m = m.rename(columns=dic)
    m_new = m[m.columns[1::2]]/1000000
    col_name = mid[name_list]
    output = pd.concat([col_name,m_new],axis = 1)
    
    return output#.groupby(name_list).sum().reset_index()

In [4]:
mid = get_mid(info) 

#replace the unmatched string to the same one, then we can merge df2 and df_new 
#without adding df3 init.
df_John.at[6371, 'CompanyBusName'] = '"TATE ASSET MANAGEMENT" AND "DCO WEALTH MANAGEMENT"'
df_John = df_John.sort_values('CompanyLegalName')

mid = mid.sort_values('LegalNm')
mid['WebAddr'] = list(df_John['Web'].str.lower())

In [5]:
mid_concatCRD = get_concatCRD_mid(mid,'BusNm')
df_John_concatCRD = get_concatCRD_John(df_John,'CompanyBusName')
#AUM_plus_WebAddr_df = get_additional_col(mid,['BusNm','WebAddr'])

In [282]:
Full_df = get_additional_col(mid,['BusNm','WebAddr','PhNb']) #14564 rows × 17 columns
df0 = Full_df.groupby(['BusNm','WebAddr','PhNb']).sum().reset_index() #12665 rows × 17 columns

df0_BusNm = df0.groupby(['BusNm']).sum().reset_index() #12622 rows × 15 columns
df0_WebAddr_PhNb_first = df0.groupby(['BusNm']).first().reset_index()[['WebAddr','PhNb']]#12622 rows × 2 columns
df1 = pd.concat([df0_WebAddr_PhNb_first,df0_BusNm],axis =1) #12622 rows × 17 columns

df1_WebAddr = df1.groupby(['WebAddr']).sum().reset_index() #12352 rows × 15 columns
df1_BusNm_PhNb_first = df1.groupby(['WebAddr']).first().reset_index()[['BusNm','PhNb']] #12352 rows × 2 columns
df2 = pd.concat([df1_BusNm_PhNb_first,df1_WebAddr],axis =1) #12352 rows × 17 columns

df2_PhNb = df2.groupby(['PhNb']).sum().reset_index() #11962 rows × 15 columns
df2_BusNm_WebAddr_first = df2.groupby(['PhNb']).first().reset_index()[['BusNm','WebAddr']] #11962 rows × 2 columns
df3 = pd.concat([df2_BusNm_WebAddr_first,df2_PhNb],axis =1) #11962 rows × 17 columns

In [248]:
#idx =list(set(list(s[s['WebAddr_x'] != s['WebAddr_y']].index) + 
#         list(s[s['PhNb_x'] != s['PhNb_y']][['PhNb_x','PhNb_y']].index)))

There are only 25 either website or phone number is unmatch from mid_concat's, then use <br/> mid_concat's website or phone number instead, which is [WebAddr_x,PhNb_x]
drop the [WebAddr_y,PhNb_y],  <br/>then we are good now for investment type classfication 

In [344]:
df_concat = pd.merge(mid_concatCRD, df3, how='inner', on=['BusNm'])
df_concat = df_concat.drop(columns=['WebAddr_y', 'PhNb_y'])
df_concat.rename(columns={'WebAddr_x': 'WebAddr', 'PhNb_x': 'PhNb'}, inplace=True)

#replace "Q5DX" with the aggregated "_AUM" value so that we can still using the origianl code to categorize the 
#investor's type
df_concat[df_concat.columns[28:56][1::2]] = df_concat[df_concat.columns[118:]]
df_concat #11962 rows × 132 columns

,BusNm,LegalNm,FirmCrdNb,Strt1,Strt2,City,State,Cntry,PostlCd,PhNb,...,Business development companies_AUM,Pooled investment vehicles_AUM,Pension and profit sharing plans_AUM,Charitable organizations_AUM,State or municipal government entities_AUM,Other investment advisers_AUM,Insurance companies_AUM,Sovereign wealth funds and foreign official institutions_AUM,Corporations or other businesses not listed above_AUM,Other_AUM
0,"""TATE ASSET MANAGEMENT"" AND ""DCO WEALTH MANAGE...","ACAS, LLC",160657,5531 CANCHA DE GOLF,SUITE 206,RANCHO SANTA FE,CA,United States,92091,888-500-9267,...,0.0,0.000000,0.000000,2.734936,0.000000,0.0,0.0,0.0,0.000000,0.0
1,&WEALTH PARTNERS,"AND WEALTH PARTNERS, LLC",149136,1629 THAMES STREET,SUITE 300,BALTIMORE,MD,United States,21231,410-844-3300,...,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0
2,"010118 MANAGEMENT, L.P.","010118 MANAGEMENT, L.P.",301237,415 JACKSON STREET,3RD FLOOR,SAN FRANCISCO,CA,United States,94111,504-320-2867,...,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0
3,"1 NORTH WEALTH SERVICES, LLC","1 NORTH WEALTH SERVICES, LLC",115948,1997 ANNAPOLIS EXCHANGE PARKWAY,SUITE 250,ANNAPOLIS,MD,United States,21401,410-975-0099,...,0.0,0.000000,0.000000,2.905368,0.000000,0.0,0.0,0.0,11.319205,0.0
4,"10-15 ASSOCIATES, INC.","10-15 ASSOCIATES, INC.",113714,9 COATES DRIVE,SUITE 3,GOSHEN,NY,United States,10924,845-294-2080,...,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11957,ZUCKERMAN INVESTMENT GROUP,"ZUCKERMAN INVESTMENT GROUP, LLC",137785,155 NORTH WACKER DRIVE,SUITE 1700,CHICAGO,IL,United States,60606,312-948-8000,...,0.0,84.697119,4.500990,7.159874,0.000000,0.0,0.0,0.0,1.363310,0.0
11958,"ZUNA, LLC","ZUNA, LLC",306113,"1000 SW BROADWAY,",SUITE 1010,PORTLAND,OR,United States,97205,971-339-5701,...,0.0,0.000000,142.321579,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0
11959,"ZURAW FINANCIAL ADVISORS, LLC","ZURAW FINANCIAL ADVISORS, LLC",173129,806 GREEN VALLEY ROAD,SUITE 201,GREENSBORO,NC,United States,27408,336. 290 7062 EXT 101,...,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0
11960,ZWEIG-DIMENNA ASSOCIATES LLC,ZWEIG-DIMENNA ASSOCIATES LLC,161385,900 THIRD AVE,NaN,NEW YORK,NY,United States,10022,212-451-1100,...,0.0,1299.396557,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0


In [345]:
# Primary
def initialize_temp(mid):
    temp = mid.iloc[:,0:11]
    temp['TtlEmp'] = mid['TtlEmp']
    temp['U.S. Dollar Discretionary AUM Amount'] = mid['Q5F2A']/1000000
    temp['U.S. Dollar Non-Discretionary AUM Amount'] = mid['Q5F2B']/1000000
    temp['U.S. Dollar Total AUM Amount'] = mid['Q5F2C']/1000000
    temp['Total Number of Discretionary AUM Accounts'] = mid['Q5F2D']
    temp['Total Number of Non-Discretionary AUM Accounts'] = mid['Q5F2E']
    temp['Total Number of AUM Account'] = mid['Q5F2F']
    temp['Total Regulatory Assets'] = mid['Q5F3']/1000000
    temp['Investor Type(Primary)'] = np.nan
    temp['Investor Type(Primary)'] = temp['Investor Type(Primary)'].astype(str)
    temp['Investor Type(Secondary)'] = np.nan
    temp['Investor Type(Secondary)'] = temp['Investor Type(Secondary)'].astype(str)
    return temp


In [346]:
def getPrimary_item7(temp,mid):
    
    for i in range(len(mid)):  
        
        primary = [] 
        
        # Item 7:
        if (np.isnan(mid.loc[i,'Q5F2C']) == True or mid.loc[i,'Q5F2C'] == 0) and mid.loc[i, 'Q7A12'] == 'Y':
            primary.append('Insurance Company')
  
        if (np.isnan(mid.loc[i,'Q5F2C']) == True or mid.loc[i,'Q5F2C'] == 0) and mid.loc[i, 'Q7A8'] == 'Y':
            #(8) banking or thrift institution
            primary.append('Bank')
     
        if (np.isnan(mid.loc[i,'Q5F2C']) == True or mid.loc[i,'Q5F2C'] == 0) and mid.loc[i, 'Q7A9'] == 'Y':
            #(9) trust company
            primary.append('Trust')
        
        Primary_List = list(set(primary))
        primary_str = ','.join([ele for ele in Primary_List])       
        
        if primary_str != '':
            #print(primary_str)
            temp.loc[i,'Investor Type(Primary)'] = primary_str
    
    return temp 

def getPrimary_item6(temp,mid):
    
    for i in range(len(mid)):          
        
        primary = []        
        
        #item 6:            
        if (np.isnan(mid.loc[i,'Q5F2C']) == True or mid.loc[i,'Q5F2C'] == 0) and mid.loc[i, 'Q6A7'] == 'Y':
            primary.append('Bank')

        if (np.isnan(mid.loc[i,'Q5F2C']) == True or mid.loc[i,'Q5F2C'] == 0) and mid.loc[i, 'Q6A8'] == 'Y':
            primary.append('Trust')
        
        if (np.isnan(mid.loc[i,'Q5F2C']) == True or mid.loc[i,'Q5F2C'] == 0) and mid.loc[i, 'Q6A13'] == 'Y':
            #13. lawyer or lawfirm
            primary.append('Service Provider - Legal')

        if (np.isnan(mid.loc[i,'Q5F2C']) == True or mid.loc[i,'Q5F2C'] == 0) and mid.loc[i, 'Q6A12'] == 'Y':
            #12. accountant or accountingfirm
            primary.append('Service Provider - Accountant')
        
        if (np.isnan(mid.loc[i,'Q5F2C']) == True or mid.loc[i,'Q5F2C'] == 0) and mid.loc[i, 'Q6A13'] == 'Y':
            #13. lawyer or lawfirm
            primary.append('Service Provider - Legal')
        
        Primary_List = list(set(primary))
        primary_str = ','.join([ele for ele in Primary_List])       
        
        if primary_str != '':
            temp.loc[i,'Investor Type(Primary)'] = primary_str
        
        #Broker/Dealer is the most important in Q6 and Q7:
        if (np.isnan(mid.loc[i,'Q5F2C']) == True or mid.loc[i,'Q5F2C'] == 0) and (mid.loc[i, 'Q6A1'] == 'Y' or mid.loc[i, 'Q7A1'] == 'Y'):
            temp.loc[i,'Investor Type(Primary)'] = 'Broker/Dealer'
            
    return temp


def getPrimary_itemQ5G(temp,mid):
       
    for i in range(len(mid)):  
        
        primary = []        
        
        #item Q5G:            
        if np.isnan(mid.loc[i,'Q5F2C']) == True or mid.loc[i,'Q5F2C'] == 0:
            if mid.loc[i,'Q5G4'] == 'Y':
                primary.append('Pooled investment vehicles')          
            if mid.loc[i,'Q5G1'] == 'Y' or mid.loc[i,'Q5G2'] == 'Y':
                primary.append('Wealth Advisors')
            if mid.loc[i,'Q5G5'] == 'Y':
                primary.append('institutional')
            if mid.loc[i,'Q5G7'] == 'Y':
                primary.append('FOF')
            if mid.loc[i,'Q5G6'] == 'Y':
                primary.append('Consultant')
            if mid.loc[i,'Q5G3'] == 'Y':
                primary.append('investment company')
            if mid.loc[i,'Q5G8'] == 'Y' or mid.loc[i,'Q5G10'] == 'Y' or mid.loc[i,'Q5G11'] == 'Y':
                primary.append('Service Provider - Research')
            if mid.loc[i,'Q5G9'] == 'Y':
                primary.append('Service Provider - Ratings')
        
        primary_str = list(set(primary))
        #primary_str = ','.join([ele for ele in Primary_List])       
        if primary_str != '':
            if len(primary_str) == 1: 
                temp.loc[i,'Investor Type(Primary)'] = primary_str
            if len(primary_str) > 1: 
                temp.loc[i,'Investor Type(Primary)'] = primary_str[0]
    
        #only need one condition based on Q2, which has a higher hierarchy than Q6 and Q7:
        if (np.isnan(mid.loc[i,'Q5F2C']) == True or mid.loc[i,'Q5F2C'] == 0) and mid.loc[i, 'Q2A11'] == 'Y':
            temp.loc[i,'Investor Type(Primary)'] = 'Wealth Advisors - Internet'
     
    return temp  


In [347]:
temp = initialize_temp(df_concat)

temp_input = temp.copy()
temp1 = getPrimary_item7(temp_input,df_concat)

temp_input1 = temp1.copy()
temp2 = getPrimary_item6(temp_input1,df_concat)

temp_input2 = temp2.copy()
temp3 = getPrimary_itemQ5G(temp_input2,df_concat)

In [348]:
temp3['Investor Type(Primary)'].value_counts()[:40]

nan                              11634
Wealth Advisors - Internet          76
Wealth Advisors                     75
Broker/Dealer                       72
Consultant                          28
FOF                                 25
institutional                       14
Pooled investment vehicles          13
Insurance Company                    8
investment company                   6
Service Provider - Research          5
Service Provider - Ratings           4
Service Provider - Accountant        1
Bank                                 1
Name: Investor Type(Primary), dtype: int64

In [352]:
def advanced_Primary(temp,mid):
    
    def find_columns(starting_word):
        c = mid.columns
        return [word for word in c if word.startswith(starting_word)]

    def func(row):
        name = mid.loc[row]['BusNm']
        f = mid[mid['BusNm'] == name]
        ff = f[find_columns('Q2A11')]
        if ff[ff == 'Y'].count(axis = 0).sum() > 0:
            return get_type(ff)
        else:
            ff = f[find_columns('Q6A')] #call the find columns function       
            if ff[ff == 'Y'].count(axis = 0).sum() > 0:
                return get_type(ff)
            else:
                ff = f[find_columns('Q7A')]
                if ff[ff == 'Y'].count(axis = 0).sum() > 0:
                    return get_type(ff)

    def get_type(df):
        col_list_Y = []
        for col in df.columns:
            if df[col].iloc[0] == 'Y':
                col_list_Y.append(col)

        dic_Q267 = {'Q2A11': 'Internet adviser', 
                    'Q6A1': 'broker-dealer', 
                    'Q6A2':'registered representative', 
                    'Q6A3':'commodity pool operator',
                    'Q6A4':'futures commission merchant',
                    'Q6A5':'real estate', 
                    'Q6A6':'insurance broker or agent', 
                    'Q6A7':'bank',
                    'Q6A8': 'trust company', 
                    'Q6A9':'registered municipal advisor', 
                    'Q6A10':'registered security-based swap dealer', 
                    'Q6A11':'major security-based swap participant',
                    'Q6A12':'accountant or accountingfirm',
                    'Q6A13':'lawyer or lawfirm',
                    'Q6A14':'other financial product salesperson', 
                    'Q7A1':'broker-dealer',
                    'Q7A2':'other investment adviser',
                    'Q7A3':'registered municipal advisor',
                    'Q7A4':'registered security-based swap dealer', 
                    'Q7A5':'major security-based swap participant',
                    'Q7A6':'commodity pool operator',
                    'Q7A7':'futures commission merchant',
                    'Q7A8':'bank',
                    'Q7A9':'trust company', 
                    'Q7A10':'accountant or accountingfirm',
                    'Q7A11':'lawyer or lawfirm', 
                    'Q7A12':'insurance company or agency',
                    'Q7A13':'pension consultant', 
                    'Q7A14':'real estate broker',
                    'Q7A15':'sponsor or syndicator of limited partnerships',
                    'Q7A16':'sponsor or general/managing member'}

        return dic_Q267[col_list_Y[0]]            
    
    
    #creating new column for summing up from g1 to n1
    def find_sum_df(mid):
        Q5D_column = list(mid.columns[28:56][1::2])
        First_six = Q5D_column[:6]
        #if n > 2:
        First_six.pop(2) #delete the bank section out of six major sections
        rest_columns = Q5D_column[6:]
        rest_columns.insert(0,'Q5DC3') # add the bank section to the rest columns

        rest_sum_col = mid[rest_columns].sum(axis = 1)
        df_withrestsum = pd.concat([mid[First_six],rest_sum_col],axis = 1).rename(columns={0:"rest_sum"})
        #True_and_False_column = df_insitiutional.max(axis = 1) == df_insitiutional['rest_sum']
        return df_withrestsum
    
    def get_client_type(a,n):
        '''
        a: df_withrestsum
        n: ith row for the for loop
        
        '''
    
        dic_client_type = {'Q5DA3': 'Wealth Advisors','Q5DB3':'Wealth Advisors','Q5DD3':'Investment',
                           'Q5DE3':'Business development','Q5DF3':'Pooled investment vehicles',
                           'rest_sum':'institutional'}
        
        return dic_client_type[a.iloc[n].index[a.iloc[n].argmax()]]
    
    df_withrestsum = find_sum_df(mid)
    
    for i in range(len(mid)):
        
        #if sum(rest) is the max in this axis, then institutional
        if df_withrestsum.max(axis = 1)[i] != 0:
            temp.loc[i,'Investor Type(Primary)'] = get_client_type(df_withrestsum,i)
        
        if 'Consultant' in temp.loc[i,'Investor Type(Primary)']:
            if mid.loc[i,'Q5F2B'] > 0.6 * mid.loc[i,'Q5F2C']:
                temp.loc[i,'Investor Type(Primary)'] = 'Consultant' 
            else:
                temp.loc[i,'Investor Type(Primary)'] = (temp.loc[i,'Investor Type(Primary)'].replace(',Consultant', '')).strip()
                #temp.loc[i,'Investor Type(Primary)'] = (temp.loc[i,'Investor Type(Primary)'].replace('Consultant', '')).strip()   
        
        if mid.loc[i, 'Q5DB1'] != 0 and not np.isnan(mid.loc[i, 'Q5DB1']):
            if mid.loc[i, 'Q5DB3'] / mid.loc[i, 'Q5DB1'] > 7.5 and ('Wealth Advisors' in temp.loc[i,'Investor Type(Primary)']) and mid.loc[i, 'Q5DB3'] > mid.loc[i, 'Q5DA3']:
                temp.loc[i,'Investor Type(Primary)'] = 'Multi Family Office'

        if (np.isnan(mid.loc[i,'Q5F2C']) == True or mid.loc[i,'Q5F2C'] == 0) and mid.loc[i, 'Q2B2'] == 'Y' and mid.loc[i, 'Q2B1'] != 'Y':
            temp.loc[i,'Investor Type(Primary)'] = 'Private Fund - less than 150M'

        if (np.isnan(mid.loc[i,'Q5F2C']) == True or mid.loc[i,'Q5F2C'] == 0) and mid.loc[i, 'Q2B1'] == 'Y':
            temp.loc[i,'Investor Type(Primary)'] = 'Venture Capital'
            
        if mid.loc[i, 'Q2B2'] == 'Y' and ('Venture Capital' in temp.loc[i,'Investor Type(Primary)']):
            temp.loc[i,'Investor Type(Primary)'] = 'Venture Capital fund - less than 150M'

        if temp.loc[i,'Investor Type(Primary)'] == 'nan' or temp.loc[i,'Investor Type(Primary)'] == '':
            #print(i)
            temp.loc[i,'Investor Type(Primary)'] = func(i)
          
   
    return temp
    
    

In [353]:
temp3_input = temp3.copy()
temp4 = advanced_Primary(temp3_input,df_concat)
sum(temp4['Investor Type(Primary)'].value_counts())

11960

In [307]:
temp3_input = temp3.copy()
temp4 = advanced_Primary(temp3_input,df_concat)
temp4['Investor Type(Primary)'].value_counts()

Wealth Advisors                                  5384
Pooled investment vehicles                       2566
institutional                                    1262
Private Fund - less than 150M                     808
Venture Capital                                   577
Investment                                        476
Multi Family Office                               382
Venture Capital fund - less than 150M             279
Wealth Advisors - Internet                         76
Consultant                                         28
Business development                               22
FOF                                                22
sponsor or general/managing member                 22
Broker/Dealer                                      15
other investment adviser                           15
commodity pool operator                             9
Service Provider - Research                         5
investment company                                  5
Service Provider - Ratings  

In [308]:
def get_secondary(temp,mid):
    
    
    for i in range(len(mid.index)):
        a1 = mid.loc[i, 'Q5DA3']
        b1 = mid.loc[i, 'Q5DB3']
        c1 = mid.loc[i, 'Q5DC3']
        d1 = mid.loc[i, 'Q5DD3']
        e1 = mid.loc[i, 'Q5DE3']
        f1 = mid.loc[i, 'Q5DF3']
        g1 = mid.loc[i, 'Q5DG3']
        h1 = mid.loc[i, 'Q5DH3']
        i1 = mid.loc[i, 'Q5DI3']
        j1 = mid.loc[i, 'Q5DJ3']
        k1 = mid.loc[i, 'Q5DK3']
        l1 = mid.loc[i, 'Q5DL3']
        m1 = mid.loc[i, 'Q5DM3']
        n1 = mid.loc[i, 'Q5DN3']
        test_list = [a1,b1,c1,d1,e1,f1,g1,h1,i1,j1,k1,l1,m1,n1]
        for j in range(len(test_list)): 
            if np.isnan(test_list[j]): 
                test_list[j] = 0
        a1 = test_list[0]
        b1 = test_list[1]
        c1 = test_list[2]
        d1 = test_list[3]
        e1 = test_list[4]
        f1 = test_list[5]
        g1 = test_list[6]
        h1 = test_list[7]
        i1 = test_list[8]
        j1 = test_list[9]
        k1 = test_list[10]
        l1 = test_list[11]
        m1 = test_list[12]
        n1 = test_list[13]   

        if h1 > 0.5 * sum(test_list) and temp.loc[i,'Investor Type(Primary)'] == 'Institutional':
            temp.loc[i,'Investor Type(Primary)'] = 'Outsourced CIO'

    # for i in range(len(mid.index)):
    #     if mid.loc[i, 'Q2A13'] == 'Y':
    #         temp.loc[i,'Investor Type(Primary)'] = 'Not eligible for registration' 

    # Secondary
    for i in range(len(mid.index)):
        sec = []
        a1 = mid.loc[i, 'Q5DA3']
        b1 = mid.loc[i, 'Q5DB3']
        c1 = mid.loc[i, 'Q5DC3']
        d1 = mid.loc[i, 'Q5DD3']
        e1 = mid.loc[i, 'Q5DE3']
        f1 = mid.loc[i, 'Q5DF3']
        g1 = mid.loc[i, 'Q5DG3']
        h1 = mid.loc[i, 'Q5DH3']
        i1 = mid.loc[i, 'Q5DI3']
        j1 = mid.loc[i, 'Q5DJ3']
        k1 = mid.loc[i, 'Q5DK3']
        l1 = mid.loc[i, 'Q5DL3']
        m1 = mid.loc[i, 'Q5DM3']
        n1 = mid.loc[i, 'Q5DN3']
        test_list = [a1,b1,c1,d1,e1,f1,g1,h1,i1,j1,k1,l1,m1,n1]
        for j in range(len(test_list)): 
            if np.isnan(test_list[j]): 
                test_list[j] = 0
        a1 = test_list[0]
        b1 = test_list[1]
        c1 = test_list[2]
        d1 = test_list[3]
        e1 = test_list[4]
        f1 = test_list[5]
        g1 = test_list[6]
        h1 = test_list[7]
        i1 = test_list[8]
        j1 = test_list[9]
        k1 = test_list[10]
        l1 = test_list[11]
        m1 = test_list[12]
        n1 = test_list[13]
        ### Secondary 
        '''
        Transferred from primary to secondary
        #(12) insurance company or agency
            if (np.isnan(mid.loc[i,'Q5F2C']) == True or mid.loc[i,'Q5F2C'] == 0) and mid.loc[i, 'Q7A12'] == 'Y':
                temp.loc[i,'Investor Type(Primary)'] = 'Insurance Company'

            # (8) banking or thrift institution
            if (np.isnan(mid.loc[i,'Q5F2C']) == True or mid.loc[i,'Q5F2C'] == 0) and mid.loc[i, 'Q7A8'] == 'Y':
                temp.loc[i,'Investor Type(Primary)'] = 'Bank'

            #    (9) trust company
            if (np.isnan(mid.loc[i,'Q5F2C']) == True or mid.loc[i,'Q5F2C'] == 0) and mid.loc[i, 'Q7A9'] == 'Y':
                temp.loc[i,'Investor Type(Primary)'] = 'Trust'

            if (np.isnan(mid.loc[i,'Q5F2C']) == True or mid.loc[i,'Q5F2C'] == 0) and mid.loc[i, 'Q5G4'] == 'Y': #and mid.loc[i, 'Q2A8'] == 'Y' 
                temp.loc[i,'Investor Type(Primary)'] = 'Private Fund'

            #1. broker-dealer (registered or unregistered)
            if (np.isnan(mid.loc[i,'Q5F2C']) == True or mid.loc[i,'Q5F2C'] == 0) and mid.loc[i, 'Q6A1'] == 'Y':
                temp.loc[i,'Investor Type(Primary)'] = 'Broker/Dealer'

            #(6) Pension consulting services    
            if (np.isnan(mid.loc[i,'Q5F2C']) == True or mid.loc[i,'Q5F2C'] == 0) and mid.loc[i, 'Q5G6'] == 'Y':
                temp.loc[i,'Investor Type(Primary)'] = 'Consultant' #Pension 

            if (np.isnan(mid.loc[i,'Q5F2C']) == True or mid.loc[i,'Q5F2C'] == 0) and mid.loc[i, 'Q2B2'] == 'Y':
                temp.loc[i,'Investor Type(Primary)'] = 'Private Fund'

            if (np.isnan(mid.loc[i,'Q5F2C']) == True or mid.loc[i,'Q5F2C'] == 0) and mid.loc[i, 'Q2B1'] == 'Y':
                temp.loc[i,'Investor Type(Primary)'] = 'Venture Capital'

            if mid.loc[i, 'Q5G6'] == 'Y' and mid.loc[i, 'Q5F2A'] > (mid.loc[i, 'Q5F2B']/2) and mid.loc[i, 'Q5DG3'] > (mid.loc[i, 'Q5F2C']/2):
                temp.loc[i,'Investor Type(Primary)'] = 'Consultant'

            if mid.loc[i, 'Q5G6'] == 'Y' and mid.loc[i, 'Q5F2A'] > (mid.loc[i, 'Q5F2C']/2):
                temp.loc[i,'Investor Type(Primary)'] = 'Consultant - Discretionary'

            if mid.loc[i, 'Q5G6'] == 'Y' and mid.loc[i, 'Q7A8'] == 'Y' and mid.loc[i, 'Q5F2A'] >  (mid.loc[i, 'Q5F2C']/2) and mid.loc[i, 'Q5DC3'] > 0:
                temp.loc[i,'Investor Type(Primary)'] = 'Bank'

            ### Secondary 
        '''    


        if mid.loc[i,'Q5F2A'] > 0 and (a1 + b1) > (sum(test_list)/2) and mid.loc[i,'Q5G12'] == 'Y':
            sec.append('FOF')

        if mid.loc[i, 'Q5G7'] == 'Y' and mid.loc[i, 'Q5G1'] == 'N':
            sec.append('FOF')
        
        #corp is not in pension and Q5DM3 col != 0:
        if mid.loc[i,'Q5G6'] != 'Y' and mid.loc[i,'Q5DM3'] != 0:
            sec.append('Corporation')

        if (mid.loc[i, 'Q5F2C'] == 0) and (mid.loc[i,'Q7A14'] == 'Y' or mid.loc[i,'Q6A5'] == 'Y'):
            sec.append('Real Estate Fund')
        
        if mid.loc[i, 'Q6A6'] == 'Y':
            sec.append('insurasnce broker or agent')    

        if mid.loc[i, 'Q6A7'] == 'Y':
            sec.append('Bank')    

        if mid.loc[i, 'Q6A8'] == 'Y':
            sec.append('Trust')

        if mid.loc[i, 'Q7B'] == 'Y':
            sec.append('Private Fund')

        if mid.loc[i,'Q5G1'] == 'Y' and mid.loc[i,'Q5G6'] == 'Y':
            sec.append('Consultant')

        if mid.loc[i,'Q5G6'] == 'Y':
            sec.append('Consultant')

        if (np.isnan(mid.loc[i,'Q5F2C']) == True or mid.loc[i,'Q5F2C'] == 0) and mid.loc[i, 'Q7A1'] == 'Y':
            sec.append('Broker/Dealer')

        if (np.isnan(mid.loc[i,'Q5F2C']) == True or mid.loc[i,'Q5F2C'] == 0) and mid.loc[i, 'Q7A2'] == 'Y':
            sec.append('Wealth Advisors')
        
        if (np.isnan(mid.loc[i,'Q5F2C']) == True or mid.loc[i,'Q5F2C'] == 0) and mid.loc[i, 'Q7A10'] == 'Y':
            sec.append('Service Provider - Ratings')

        if (np.isnan(mid.loc[i,'Q5F2C']) == True or mid.loc[i,'Q5F2C'] == 0) and mid.loc[i, 'Q6A13'] == 'Y':
            sec.append('Service Provider - Legal')

        if (np.isnan(mid.loc[i,'Q5F2C']) == True or mid.loc[i,'Q5F2C'] == 0) and mid.loc[i, 'Q6A12'] == 'Y':
            sec.append('Service Provider - Accountant')

        if (np.isnan(mid.loc[i,'Q5F2C']) == True or mid.loc[i,'Q5F2C'] == 0) and mid.loc[i, 'Q5G9'] == 'Y':
            sec.append('Service Provider - Ratings')

        if (np.isnan(mid.loc[i,'Q5F2C']) == True or mid.loc[i,'Q5F2C'] == 0) and (mid.loc[i, 'Q5G8'] == 'Y' or mid.loc[i, 'Q5G11'] == 'Y' or mid.loc[i, 'Q5G10'] == 'Y'):
            sec.append('Service Provider - Research')

        if (np.isnan(mid.loc[i,'Q5F2C']) == True or mid.loc[i,'Q5F2C'] == 0) and mid.loc[i, 'Q7A12'] == 'Y':
            sec.append('Insurance Company')

        if (np.isnan(mid.loc[i,'Q5F2C']) == True or mid.loc[i,'Q5F2C'] == 0) and mid.loc[i, 'Q7A8'] == 'Y':
            sec.append('Bank')

        if (np.isnan(mid.loc[i,'Q5F2C']) == True or mid.loc[i,'Q5F2C'] == 0) and mid.loc[i, 'Q7A9'] == 'Y':
            sec.append('Trust')

        if (np.isnan(mid.loc[i,'Q5F2C']) == True or mid.loc[i,'Q5F2C'] == 0) and mid.loc[i, 'Q5G4'] == 'Y':
            sec.append('Private Fund')

        if (np.isnan(mid.loc[i,'Q5F2C']) == True or mid.loc[i,'Q5F2C'] == 0) and mid.loc[i, 'Q6A1'] == 'Y':
            sec.append('Broker/Dealer')

        if (np.isnan(mid.loc[i,'Q5F2C']) == True or mid.loc[i,'Q5F2C'] == 0) and mid.loc[i, 'Q5G6'] == 'Y':
            sec.append('Consultant')
            
        if (np.isnan(mid.loc[i,'Q5F2C']) == True or mid.loc[i,'Q5F2C'] == 0) and mid.loc[i, 'Q7A11'] == 'Y':
            sec.append('lawyer or lawfirm')

        if mid.loc[i, 'Q5G6'] == 'Y' and mid.loc[i, 'Q5F2A'] > (mid.loc[i, 'Q5F2B']/2) and mid.loc[i, 'Q5DG3'] > (mid.loc[i, 'Q5F2C']/2):
            sec.append('Consultant')

          
        if mid.loc[i, 'Q5G6'] == 'Y':# and mid.loc[i, 'Q5F2A'] > 0.6 * (mid.loc[i, 'Q5F2C']):
            sec.append('Consultant - Discretionary')

        if mid.loc[i, 'Q5G6'] == 'Y' and mid.loc[i, 'Q7A8'] == 'Y' and mid.loc[i, 'Q5F2A'] >  (mid.loc[i, 'Q5F2C']/2) and mid.loc[i, 'Q5DC3'] > 0:
            sec.append('Bank')

        if f1 > (sum(test_list))/2:
            sec.append('Private Fund')

        if d1 > (sum(test_list))/2:
            sec.append('Mutual Fund')

        if (d1 + f1) >= max(test_list):
            if d1 > f1:
                sec.append('Mutual Fund')
            elif f1 > d1:
                sec.append('Private Fund')

        if ((d1 + f1) > max(test_list) or (d1 + f1) > 0.75* max(test_list)) and d1 != 0 and f1 != 0 and d1 > 0.33*(d1 + f1) and f1 > 0.33*(d1 + f1):
            if d1 > f1:
                sec.append('Multi-strategy Mutual')
            elif d1 < f1:
                sec.append('Multi-strategy Alternative')

        if e1 != 0 and a1 == 0 and b1 == 0 and c1 == 0 and d1 == 0 and f1 == 0 and g1 == 0 and h1 == 0 and i1 == 0 and j1 == 0 and k1 == 0 and l1 == 0 and m1 == 0:
            sec.append('BDC')

        if max(test_list) == c1 and mid.loc[i, 'Q7A9'] == 'Y' and c1 != 0:
            sec.append('Bank-Trust')

        if k1 > (sum(test_list)/2) and k1 != 0:
            sec.append('Insurance Company')

        if (mid.loc[i,'Q5G1'] == 'N' and (a1 + b1) > sum(test_list)*0.65) or (mid.loc[i,'Q5G1'] == 'Y' and (a1 + b1) > sum(test_list)*0.3):
            sec.append('Wealth Advisors')

        '''
        if mid.loc[i, 'Q5DB1'] != 0 and not np.isnan(mid.loc[i, 'Q5DB1']):
            if mid.loc[i, 'Q5DB3'] / mid.loc[i, 'Q5DB1'] > 75000000 and temp.loc[i,'Investor Type(Primary)'] == 'Wealth Advisors' and mid.loc[i, 'Q5DB3'] > 0.5*(mid.loc[i, 'Q5DB3'] + mid.loc[i, 'Q5DA3']):
                sec.append('Multi Family Office')
        '''

        if (mid.loc[i, 'Q5DA3']+mid.loc[i, 'Q5DB3']) > 0.75*(mid.loc[i, 'Q5DD3']+mid.loc[i, 'Q5DF3']) and (mid.loc[i, 'Q5DA3'] + mid.loc[i, 'Q5DB3']) > 1/1000000 * 0.33 * mid.loc[i, 'Q5F2C']:
            sec.append('Wealth Advisors')
        elif (mid.loc[i, 'Q5DA3'] + mid.loc[i, 'Q5DB3']) > 1/1000000 * 0.67 * mid.loc[i, 'Q5F2C']:
            sec.append('Wealth Advisors')

        if temp.loc[i,'Investor Type(Primary)'] == 'Bank' and mid.loc[i, 'Q5G6'] == 'Y':
            sec.append('Pension Consultant')

        if h1 > 0.5 * sum(test_list) and temp.loc[i,'Investor Type(Primary)'] == 'Institutional':
            sec.append('Outsourced CIO')
        
        #Addtional looser conditions to try to fill more nan cells:
        if mid.loc[i, 'Q5DB3'] / mid.loc[i, 'Q5DB1'] > 7.5 and ('Wealth Advisors' not in temp.loc[i,'Investor Type(Primary)']):
            sec.append('Multi Family Office')
        
        #check with Dianna:
        if (a1*b1) != 0:
            sec.append('Wealth Advisors')
        
        if c1 != 0:
            sec.append('Bank')
        
        if d1 != 0:
            sec.append('Mutual Fund')
        
        if e1 != 0:
            sec.append('Business Development') 
        
        if f1 != 0:
            sec.append('Private Fund')
        
        if g1 != 0:
            sec.append('Corp Pension')
        
        if h1 != 0:
            sec.append('Endowment')
        
        if i1 !=0:
            sec.append('Pension-State')
        
        if j1 !=0:
            sec.append('Other Inv Advisor')
        
        if k1 !=0:
            sec.append('Insurance')
            
        if m1 !=0:
            sec.append('Corp')
        
        if n1 !=0:
            sec.append('Other')
    
        #looser conditions end 
        ##########################

        
        secList = list(set(sec))
        
        if temp.loc[i,'Investor Type(Primary)'] in secList:
            secList.remove(temp.loc[i,'Investor Type(Primary)'])
                #remove private fund in secondary if primary has relative private fund type
        
        if temp.loc[i,'Investor Type(Primary)'] == 'Venture Capital':
            
            if 'Private Fund' in secList:
                secList.remove('Private Fund')

        secondary_str = ','.join([ele for ele in secList])
        temp.loc[i,'Investor Type(Secondary)'] = secondary_str
    
    return temp 


#temp.to_pickle("firmsData.pkl")



In [309]:
temp4_input = temp4.copy()
temp5 = get_secondary(temp4_input,df_concat)

/Users/guapofish/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:256: RuntimeWarning: invalid value encountered in double_scalars
/Users/guapofish/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:256: RuntimeWarning: divide by zero encountered in double_scalars


In [323]:
def detect_empty(temp,mid):
    idx = []
    for i in range(len(mid.index)):
        try:
            len(temp.loc[i,'Investor Type(Primary)'])
        except:
            if len([temp.loc[i,'Investor Type(Secondary)']]) == 1:
                temp.loc[i,'Investor Type(Primary)'] = temp.loc[i,'Investor Type(Secondary)']
                idx.append(i)
               
    return temp.iloc[idx].transpose()
                    

In [324]:
temp5_input = temp5.copy()
empty_primary_temp = detect_empty(temp5_input,df_concat)

In [327]:
empty_primary_temp.to_csv('/Users/guapofish/Desktop/empty_primary_temp_1023.csv')

In [328]:
dff = pd.concat([temp5,df_concat[df_concat.columns[118:]]],axis = 1)

In [331]:
dff['Investor Type(Primary)'].value_counts()

Wealth Advisors                                  5384
Pooled investment vehicles                       2566
institutional                                    1262
Private Fund - less than 150M                     808
Venture Capital                                   577
Investment                                        476
Multi Family Office                               382
Venture Capital fund - less than 150M             279
Wealth Advisors - Internet                         76
Consultant                                         28
Business development                               22
FOF                                                22
sponsor or general/managing member                 22
Broker/Dealer                                      15
other investment adviser                           15
commodity pool operator                             9
Service Provider - Research                         5
investment company                                  5
Service Provider - Ratings  

In [276]:
dff[['U.S. Dollar Discretionary AUM Amount',
       'U.S. Dollar Non-Discretionary AUM Amount',
       'U.S. Dollar Total AUM Amount',
       'Total Number of Discretionary AUM Accounts',
       'Total Number of Non-Discretionary AUM Accounts',
       'Total Number of AUM Account', 'Total Regulatory Assets']]

,U.S. Dollar Discretionary AUM Amount,U.S. Dollar Non-Discretionary AUM Amount,U.S. Dollar Total AUM Amount,Total Number of Discretionary AUM Accounts,Total Number of Non-Discretionary AUM Accounts,Total Number of AUM Account,Total Regulatory Assets
0,171.963487,2.664787,174.628274,569.0,6.0,575.0,0.600000
1,104.044440,147.030147,251.074587,169.0,49.0,218.0,0.000000
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,198.671581,0.000000,198.671581,614.0,0.0,614.0,0.000000
4,895.296047,0.000000,895.296047,2190.0,0.0,2190.0,0.000000
...,...,...,...,...,...,...,...
11957,875.006834,71.461625,946.468459,869.0,20.0,889.0,1.715521
11958,1.873979,140.447600,142.321579,1.0,5.0,6.0,0.000000
11959,238.266817,0.000000,238.266817,598.0,0.0,598.0,0.000000
11960,1299.396557,0.000000,1299.396557,4.0,0.0,4.0,28.172006


In [332]:
def get_muti_type_primary(x):
    idx = []
    for i in range(len(x)):
        try: 
            if len(x['Investor Type(Primary)'].iloc[i].split(",")) > 1:
                idx.append(i)
        except:
            idx.append(i)
            
    return x.iloc[idx]

In [333]:
muti_type_primary_df = get_muti_type_primary(dff)

In [334]:
muti_type_primary_df['Investor Type(Primary)']

4760    None
9771    None
Name: Investor Type(Primary), dtype: object

In [335]:
dff.to_csv('/Users/guapofish/Desktop/output_IAPDfirm_updated_David_1023.csv')